# ADM Homework 5 - USA Airport Flight Analysis

### Import Dataset and Libraries

In [1]:
!pip install kaggle

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!kaggle datasets download flashgordon/usa-airport-dataset

Dataset URL: https://www.kaggle.com/datasets/flashgordon/usa-airport-dataset
License(s): CC0-1.0




  0%|          | 0.00/40.8M [00:00<?, ?B/s]
  2%|▏         | 1.00M/40.8M [00:00<00:28, 1.48MB/s]
  5%|▍         | 2.00M/40.8M [00:00<00:14, 2.90MB/s]
  7%|▋         | 3.00M/40.8M [00:00<00:10, 3.94MB/s]
 10%|▉         | 4.00M/40.8M [00:01<00:08, 4.81MB/s]
 12%|█▏        | 5.00M/40.8M [00:01<00:06, 5.50MB/s]
 15%|█▍        | 6.00M/40.8M [00:01<00:06, 5.50MB/s]
 17%|█▋        | 7.00M/40.8M [00:01<00:06, 5.42MB/s]
 20%|█▉        | 8.00M/40.8M [00:01<00:06, 5.51MB/s]
 22%|██▏       | 9.00M/40.8M [00:01<00:05, 5.93MB/s]
 24%|██▍       | 10.0M/40.8M [00:02<00:04, 6.53MB/s]
 27%|██▋       | 11.0M/40.8M [00:02<00:04, 6.59MB/s]
 29%|██▉       | 12.0M/40.8M [00:02<00:04, 6.56MB/s]
 32%|███▏      | 13.0M/40.8M [00:02<00:04, 6.87MB/s]
 34%|███▍      | 14.0M/40.8M [00:02<00:04, 6.55MB/s]
 37%|███▋      | 15.0M/40.8M [00:02<00:04, 5.80MB/s]
 39%|███▉      | 16.0M/40.8M [00:03<00:04, 5.30MB/s]
 42%|████▏     | 17.0M/40.8M [00:03<00:05, 4.64MB/s]
 44%|████▍     | 18.0M/40.8M [00:03<00:05, 4.52MB/s]
 

In [3]:
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

We unzip the file downloaded from Kaggle and save it to our current working directory.

In [5]:
# Unzip the file we downloaded from kaggle
zip_file_path = 'usa-airport-dataset.zip'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('.') # extract files to current working directory

Now we save the data in a CSV file:

In [6]:
df = pd.read_csv('Airports2.csv')
df.head()

,Origin_airport,Destination_airport,Origin_city,Destination_city,Passengers,Seats,Flights,Distance,Fly_date,Origin_population,Destination_population,Org_airport_lat,Org_airport_long,Dest_airport_lat,Dest_airport_long
0,MHK,AMW,"Manhattan, KS","Ames, IA",21,30,1,254,2008-10-01,122049,86219,39.140999,-96.670799,NaN,NaN
1,EUG,RDM,"Eugene, OR","Bend, OR",41,396,22,103,1990-11-01,284093,76034,44.124599,-123.211998,44.254101,-121.150002
2,EUG,RDM,"Eugene, OR","Bend, OR",88,342,19,103,1990-12-01,284093,76034,44.124599,-123.211998,44.254101,-121.150002
3,EUG,RDM,"Eugene, OR","Bend, OR",11,72,4,103,1990-10-01,284093,76034,44.124599,-123.211998,44.254101,-121.150002
4,MFR,RDM,"Medford, OR","Bend, OR",0,18,1,156,1990-02-01,147300,76034,42.374199,-122.873001,44.254101,-121.150002
